# Accelerated convolution with silx

Since version 0.11, silx comes with a Convolution operator leveraging CPU and GPU execution parallelism.  
This convolution is computed in the "real" domain (i.e does not involve Fourier transforms).

## 0. Brief recall on terminology and boundaries handling modes

You can skip this section if you are already familiar with convolution and boundary handling.

### 0.1 - Terminology

In this tutorial, we aim at computing convolutions between two arrays. These arrays are possibly multi-dimensional (1, 2 or 3). 
The first array involved will be called **signal** (it can be a time series, an image or a volume). 
The second array will be called **kernel**. It is assumed to be smaller in size than the signal. 
The number of dimensions of the signal and kernel are denoted by $N_s$ and $N_k$, respectively.

For example, in the case of a Gaussian blur or an image, the signal is the image (2D: $N_s = 2$), and the kernel is a (separable) truncated Gaussian distribution (1D: $N_k = 1$). 

Lastly, the number of dimensions of the kernel is always smaller than the number of dimensions of the signal (that is, $N_k \leq N_s$). It does make sense to convolve a 3D volume with a 2D image, but the opposite does not.

### 0.2 - Boundaries handling

When computing a convolution, an issue arises at the "edges" of the involved signal: the convolution kernel spills outside of the signal support. 
When the kernel is near the edges, some of the signal values can be multiplied with the corresponding kernel values, while some might not. 
To overcome this, the kernel values are multiplied with "dummy" signal values, which can be:
  - Zeros: it is equivalent of ignoring the kernel values outside of the signal edges (this is what is done in `numpy.convolve(sig, kern, mode="same")`)
  - Constants (a generalization of the previous case)
  - Signal values

In the latter case, the signal values used outside of the signal support depend on the boundary handling scheme:
  - Periodic: this is equivalent at tiling the signal multiple time before applying the convolution kernel (convolution performed with FFTs assume a periodic boundaries handling).
  - Mirror, symmetric: the signal values outside of the signal support are those near the edges, "mirrored".
  - Edges: the signal values outside the signal support are those of the signal edges

The convolution in silx is designed to be compatible with the one of `scipy`. 
Please see the documentation of `scipy.ndimage.convole` for a more detailed description of each mode.


## 1. Creating a `Convolution` object

In silx, the `Convolution` object is a typical case of `OpenclProcessing`: 
  - Create an OpenCL context and processing queue. The context is created on the "best" found device, unless told otherwise
  - Register all the information relevant to the convolution processing (signal and kernel dimensions and sizes, axes, etc).
  - Compile the relevant OpenCL kernels and pre-allocate memory
  
All of this is done by specifying the information on the signal and kernel.

In [ ]:
from silx.opencl.convolution import Convolution, gaussian_kernel
from scipy.misc import ascent
img = ascent().astype("f")
g = gaussian_kernel(1.0)
C = Convolution(img.shape, g) 


All you need to provide is the shape of the signal, and the kernel array. 

Once built, the `Convolution` object can provide you with useful information.

In [ ]:
print(C.use_case_desc)

Here, the "use case" is the convolution of a 2D signal (image) with a 1D kernel.  
This is what is done by default when building the Convolution operator. A separable convolution corresponds to a convolution along the horizontal axis, followed by a convolution along the vertical axis (or the other way around).

## 2. Specifying axes: non-separable, batched, and separable convolution

The `axes` keyword argument of `Convolution` lets you choose the axes along which the convolution is computed. 
In the previous example, a one-dimensional kernel is used on a 2D image as a separable convolution.

In [ ]:
print(C.axes)

Here, `(0, 1)` means that the image is convolved with the 1D kernel along the dimension 0, then along the dimension 1.  
If the convolution is applied on only one axis (out of $N_s$) of the signal, this means that a batched convolution is computed. For example:
  - batched 1D convolution on 2D data along rows
  - batched 1D convolution on 3D data
  - batched 2D convolution on 3D data

In the following, we build the convolution object with `axes=(1,)`, so that the convolution will be performed row by row.

In [ ]:
C1 = Convolution(img.shape, g, axes=(1,))

From the signal/kernel dimensions and the axes, the object is able to infer the "use case" :

In [ ]:
print(C1.use_case_desc)

Let us compare with `scipy`.

In [ ]:
import numpy as np
from scipy.ndimage import convolve1d
ref = convolve1d(img, g, axis=1)
res = C1(img)
print(np.allclose(res, ref))

If the kernel number of dimensions ($N_k$) equals the signal number of dimensions ($N_s$), it is inferred that the convolution is not separable (the kernel is separable if its matrix rank is 1).


In [ ]:
k2 = np.array([[1.,2,3],[0,1,2],[0,1,2]])
k2 /= k2.sum()
C2 = Convolution(img.shape, k2)
print(C2.use_case_desc)

Again we can comare the convolution results with `scipy`:

In [ ]:
from scipy.ndimage import convolve
ref2 = convolve(img, k2)
res2 = C2(img)
print(np.allclose(res2, ref2))

## 3. Boundary handling

`silx.opencl.convolution.Convolution` comes with several boundary handling methods: "reflect", "nearest", "wrap", "constant".

In [ ]:
C = Convolution(img.shape, g, axes=(1,), mode="wrap")
res = C(img)
ref = convolve1d(img, g, axis=1, mode="wrap")
print(np.allclose(res,ref))

## 4. Using silx convolution in performance-critical applications

`silx.opencl.convolution` tries to be as fast as possible. The underlying implementation is done in OpenCL to exploit GPU or CPU parallelism.  
Like any `OpenclProcessing` class, the usual usage of `Convolution` is the following:
  - Build a `Convolution` object *once*
  - Apply it *many* times

The `Convolution` operator may be called on an input which is a `pyopencl.array.Array`. 
This is useful when you have some data already in GPU memory and you don't want to do extra CPU-GPU transfers. 
The output of the convolution processing may also be provided, again as a `pyopencl.array.Array`.

In [ ]:
import pyopencl.array as parray
C = Convolution(img.shape, g)
# Create a "dataset" of 100 frames on GPU.
# Then, apply a separable convolution on each frame
# (This might also be done by stacking the frames series as a 3D array !)
N = 100
# Simulate data acquisition
dummy_gpu_data = []
gpu_outputs = []
for i in range(N):
    dummy_gpu_data.append(
        parray.to_device(C.queue, img + i) 
    )
    gpu_outputs.append(
        parray.zeros_like(dummy_gpu_data[-1])
    )
# Data processing
i = 0
for gpu_frame, gpu_output in zip(dummy_gpu_data, gpu_outputs):
    print("Processing frame %d" % i)
    # The object "C" is applied multiple time, on a series of images.
    # Both input and output images are on device, so no extraneous transfer is done.
    C(gpu_frame, output=gpu_output)
    i += 1